## 目标：
### 1、掌握随机森林分类器
### 2、掌握GridSearchCV工具，优化算法模型的参数
### 3、使用Pipeline管道机制进行流水线作业

# 随机森林
### Random Forest，简写RF，实际上是一个包含多个决策树的分类器，每一个子分类器都是一颗CART分类回归树。
### 随机森林既可做分类，也可做回归。
### 1、当做分类的时候，输出结果是每个子分类器的分类结果最多的那个。可以理解为每个分类器都做投票，取投票最多的那个结果。
### 2、当做回归的时候，输出结果是每颗CART树的回归结果的平均值。

### 在sklearn中，使用RandomForestClassifier()构造随机森林模型，常用构造参数：
* n_estimators：随机森林里决策树的个数，默认是10
* criterion：决策树分裂标准，默认是基尼系数（CART算法），也可以选择entropy（ID3算法）
* max_depth：决策树的最大深度，默认是None，也就是不限制决策树的深度。也可以设置一个整数，限制决策树的最大深度。
* n_jobs：拟合和预测的时候CPU的核数，默认是1，也可以是整数。如果是-1，则代表CUP的核数。

## 使用GridSearchCV工具对模型参数进行调优
### 在做分类算法的时候，我们需要经常调节网络参数（对应上面的构造参数），目的是得到更好的分类结果。
### from sklearn.model_selection import GridSearchCV
### 使用GridSearchCV(estimator,param_grid,cv=None,scoring=None)构造参数的字段搜索模块，参数说明如下：
* estimator：代表我们想采用的分类器，比如随机森林、决策树、SVM、KNN等
* param_grid：代表我们想要优化的参数及取值，输入的是字典或列表形式
* cv：交叉验证的折数，默认None，代表使用三折交叉验证。(You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.)---给cv指定一个值而不是依赖默认值，在0.22版本中，默认值将从3更改为5

* scoring：准确度的评价标准，默认None，也是需要使用score函数。也可以设置具体的评价标准，比如accuray，f1等

### 例子：使用sklearn自带的IRIS数据集，采用随机森林对IRIS数据分类
### 假设想知道n_estimators在1-10的范围内取哪个值的分类结果最好

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import load_iris

In [2]:
# 构造随机森林
rf = RandomForestClassifier()
# 设置参数范围
parameters = {'n_estimators':range(1,11)}
# sklearn自带IRIS数据集
iris = load_iris()
#使用GridSearchCV进行参数调优
clf = GridSearchCV(estimator=rf,param_grid=parameters,cv = 5)
# 对iris数据集进行分类
clf.fit(iris.data,iris.target)
print('准确率：%.4f' %clf.best_score_)
print('最优参数：',clf.best_params_)
# 当n_estimators = 5的时候，准确率为0.9733，也就是随机森林一共有6个子决策树

准确率：0.9667
最优参数： {'n_estimators': 3}


## 使用Pipeline管道机制进行流水线作业
### python有一种管道机制，让我们把每一步都按顺序列下来，从而创建Pipeline流水线作业。每一步都采用（‘名称’，步骤）的方式表示。
### 首先需要采用StandardScaler方法对数据规范化，即采用数据规范化为均值为0，方差为1的正态分布，然后采用PCA（一种常用的降维方法）对数据进行降维，最后采用随机森林进行分类。

In [24]:
# 具体代码
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler        #规范化，使各特征的均值为1，方差为0
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
        ('scaler',StandardScaler()),
        ('pca',PCA()),
        ('randomforestclassifier',RandomForestClassifier())
    ])

### 采用Pipeline管道机制，用随机森林对IRIS数据集做分类

In [3]:
from sklearn.ensemble import RandomForestClassifier  
from sklearn.model_selection import GridSearchCV     
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [5]:
# 构造随机森林
rf = RandomForestClassifier()
parameters = {'randomforestclassifier__n_estimators':range(1,11)}
iris = load_iris()
# 采用pipeline管道机制
pipeline = Pipeline([
        ('scaler',StandardScaler()),
        ('randomforestclassifier',rf)
    ])
# 使用gridsearchcv进行参数调优
clf = GridSearchCV(estimator = pipeline,param_grid=parameters,cv = 5)
# 对iris数据集进行分类
clf.fit(iris.data,iris.target)
print('准确率：%.4f' %clf.best_score_)
print('最优参数：',clf.best_params_)

准确率：0.9733
最优参数： {'randomforestclassifier__n_estimators': 9}
